In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
import SimpleITK as sitk
import itk

In [79]:
def add_z_pt_above_below(pt,z_offset,pt_list):
    above = [pt[0],pt[1],pt[2]+z_offset]
    below = [pt[0],pt[1],pt[2]-z_offset]
    pt_list.append(below)
    pt_list.append(above)
    pt_list.append(pt)
    return pt_list

In [80]:
def validate_pt(pt,im):
    if pt[0]<0:
        pt[0] = 0
    if pt[0]>im.GetSize()[0]:
        pt[0] = im.GetSize()[0]
    if pt[1] < 0:
        pt[1] = 0
    if pt[1] > im.GetSize()[1]:
        pt[1] = im.GetSize()[1]
        
    return pt

In [81]:
def get_min_max(pts):
    min_x = 9999
    max_x = 0
    min_y = 9999
    max_y = 0
    
    for pt in pts:
        if pt[0] < min_x:
            min_x = pt[0]
        if pt[0] > max_x:
            max_x = pt[0]
        if pt[1] < min_y:
            min_y = pt[1]
        if pt[1] > max_y:
            max_y = pt[1]
    return min_x,max_x,min_y,max_y

In [82]:
def get_real_foreground_im(bb_pts,masked_im):
    # get actual ROI as foreground
    pt_list=[]
    pt_list = add_z_pt_above_below(bb_pts[0],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[1],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[2],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[3],0.5,pt_list)

    surf=Surface(pts=pt_list).create_delunay_surface_3d(return_hull=True,output_filename=output_path+'bla.vtk')

    np_im = sitk.GetArrayFromImage(masked_im)
    nonzeros = np.where(np_im>0)
    inds_pts = np.array([nonzeros[0],nonzeros[1],np.zeros_like(nonzeros[0])]).transpose()

    l = Landmarks(pts=inds_pts)
    selected = l.get_landmarks_within_given_surface(surf)
    print(selected)
    
    return selected

In [83]:
def get_bounding_plane(cube,z_coord,tx_mat,im,neg_x=False,coronal=False):
    
    intersectoin_plane = Surface(polydata=cube.get_intersection_plane(z_coord))
    
    if intersectoin_plane.surface.GetNumberOfPoints() > 3:
        
        
        if neg_x:
            neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
            intersectoin_plane.apply_transformation(neg_mat,inverse=True)
        if coronal:
            coronal_tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
            intersectoin_plane.apply_transformation(coronal_tr_mat,inverse=True)
        intersectoin_plane.write_surface_mesh(output_path+'{}_sec_cutting_plane_before.vtk'.format(sec_num))
        intersectoin_plane.apply_transformation(tx_mat,inverse=True)
        intersectoin_plane.write_surface_mesh(output_path+'{}_sec_cutting_plane_after.vtk'.format(sec_num))
#         intersectoin_plane_cube = Surface(polydata=intersectoin_plane.create_delunay_surface_3d(return_hull=True,\
#                                                              output_filename=output_path+'vS1_plane.vtk'))
        
        bounds = intersectoin_plane.surface.GetBounds()
        #pt_list = []
        
        #vtkpts = intersectoin_plane.surface.GetPoints()
        #print(selected)
        #XY_RES = 1
         
#         pt1 = [int(vtkpts.GetPoint(0)[0]/XY_RES),int(vtkpts.GetPoint(0)[1]),0]
#         pt2 = [int(vtkpts.GetPoint(1)[0]/XY_RES),int(vtkpts.GetPoint(1)[1]),0]
#         pt3 = [int(vtkpts.GetPoint(2)[0]/XY_RES),int(vtkpts.GetPoint(2)[1]),0]
#         pt4 = [int(vtkpts.GetPoint(3)[0]/XY_RES),int(vtkpts.GetPoint(3)[1]),0]
        
        # get im inds within the BB of ROI
#         if neg_x:
#             pt1 = validate_pt([-int(bounds[0]/XY_RES),int(bounds[2]/XY_RES),0],im)
#             pt2 = validate_pt([-int(bounds[1]/XY_RES),int(bounds[3]/XY_RES),0],im)
#         else:
        pt1 = validate_pt([int(bounds[0]/XY_RES),int(bounds[2]/XY_RES),0],im)
        pt2 = validate_pt([int(bounds[1]/XY_RES),int(bounds[3]/XY_RES),0],im)
        #pt3 = validate_pt(pt3,im)
        #pt4 = validate_pt(pt4,im)
        
        return [pt1,pt2]
    else:
        return None

In [84]:
def mask_im_region_from_bb(np_im,bb_pts):
    #np_im = sitk.GetArrayFromImage(im)
    #np_im[:,:] = 0
    
    min_x,max_x,min_y,max_y = get_min_max(bb_pts)
    print(min_x,max_x,min_y,max_y)
    np_im[bb_pts[0][1]:bb_pts[1][1],bb_pts[0][0]:bb_pts[1][0]] = 255
    masked_im = sitk.GetImageFromArray(np_im)
    #sitk.WriteImage(masked_im,'{}_binary_masked.tif'.format(op_filename))
    
    return np_im

In [85]:
def mark_binary_im(masked_im):
    contour_im = sitk.BinaryContour(masked_im,backgroundValue=0,foregroundValue=255)
    dil = sitk.BinaryDilateImageFilter()
    dil.SetKernelRadius(10)
    dil.SetBackgroundValue(0)
    dil.SetForegroundValue(255)
    contour_dilated = dil.Execute(contour_im)
    marked_im = sitk.Or(im,contour_dilated)
    return marked_im
    

In [86]:
def get_cylinder(radius,edge):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(10)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return cyl_surf

In [89]:
animal_name = 'MG50_lhs'
image_name = 'MG50_3Day_lhs'
section_sg_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Original_Data/Spatial_Graphs/misc/Section_Graphs/{}.am/'.format(animal_name)

tx_file = section_sg_path +'aligntobarrel.hx'
file_pattern = '*S*_ct.am'
images_path = '/nas1/Data_aman/00_Rabies/{}/00_Images/00_Confocal/ch_00_stacks/'.format(image_name)
image_file_pattern = 'S*_00.tif'
#rabies_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V4/0_Inputs/SpatialGraphs/Section_Graphs/Rabies/{}/rabies/'.format(animal_name)
#neun_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Landmarks/Manual/Rabies/NeuN/{}/'.format(animal_name)

#surface_path_vS1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_s1_hull.vtk'.format(animal_name)
surface_path_vM1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_vM1.vtk'.format(animal_name)
central_axis_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Axis_Fields/{}_vM1_PCA_2.am'.format(animal_name)
central_axis_path_neg = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Axis_Fields/{}_vM1_PCA_2_Neg.am'.format(animal_name)

#output_path = '/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_Central_Axis_Radius_250um/{}/'.format(animal_name)
output_path = '/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count_Central_Axis_Radius_250um_2D/{}/'.format(animal_name)

XY_RES = 0.868
z_cood_offset = 50
cylinder_radius = 250

In [94]:
#sg_3d = AmiraSpatialGraph(glob.glob(path + file_pattern)[0],read_header_only=True)
#landmarks3d = Landmarks()
#landmarks3d_neun = Landmarks()

#vs1_hull = Surface(surface_path_vS1)
#s1_cube = Surface(polydata=s1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                             output_filename=output_path+'vS1_Cube.vtk'))

#vm1_hull = Surface(surface_path_vM1)
#vm1_cube = Surface(polydata=vm1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                               output_filename=output_path+'vM1_Cube.vtk'))
#write_cube(vm1_cube.surface,output_path+'vM1_Cube.vtk')
vm1_central_axis_sg = AmiraSpatialGraph(central_axis_path,generic_graph=True)
vm1_central_axis_sg_neg = AmiraSpatialGraph(central_axis_path_neg,generic_graph=True)

central_cylinder = get_cylinder(cylinder_radius,[vm1_central_axis_sg_neg.graph_data.edge_list[0][1],vm1_central_axis_sg.graph_data.edge_list[0][1]])
central_cylinder.write_surface_mesh(output_path+'Cylinder.vtk')

for file in sorted(glob.glob(images_path + image_file_pattern)):
    #sg = AmiraSpatialGraph(file,axis_directions=[1,1,1])
    #print(file,tx_file)
    
    # read tx mat from the file
    with open(tx_file,'r') as f:
        lines = f.readlines()
        
    sec_num = int(os.path.basename(file)[1:-7])
    #sec_num = 40
    print(sec_num)
    
    str_to_compare = '"S{:03d}_ct.am" setTransform '.format(sec_num)
    #print(str_to_compare)
    tx_mat = []
    tx_mat_np = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).reshape(4,4)
    for line in lines:
        if line.startswith(str_to_compare):
            #tx_mat = (line[len(str_to_compare):-1].split(' '))
            for num in (line[len(str_to_compare):-1].split(' ')):
                tx_mat.append(float(num))
            tx_mat_np = np.array(tx_mat).reshape(4,4)
            break
    #print(tx_mat)
    
    if os.path.exists(images_path+'S{}_00.tif'.format(sec_num)):
        im = sitk.ReadImage(images_path+'S{}_00.tif'.format(sec_num))
        npim = sitk.GetArrayFromImage(im)
        npim[:] = 0
        # get s1 and m1 bounding box for this plane
        #s1_bb_pts = get_bounding_plane(vs1_hull,(sec_num-1) * z_cood_offset,tx_mat_np,im, neg_x=True,coronal=False)
        m1_bb_pts = get_bounding_plane(central_cylinder,(sec_num-1) * z_cood_offset,tx_mat,im, neg_x=False,coronal=False)
        #print(s1_bb_pts)
        print(m1_bb_pts)
        #Landmarks(pts=[s1_bb_pts[0]]).write_landmarks(output_path+'{}_lower.landmarksAscii'.format(sec_num))
        #Landmarks(pts=[s1_bb_pts[1]]).write_landmarks(output_path+'{}_upper.landmarksAscii'.format(sec_num))
        
        if m1_bb_pts is not None :
            with open(output_path+'S{}_length_width.txt'.format(sec_num),'w') as f:
                f.write('x={}, y={}'.format(abs(m1_bb_pts[0][0]-m1_bb_pts[1][0]),abs(m1_bb_pts[0][1]-m1_bb_pts[1][1])))
        
            npim = mask_im_region_from_bb(npim,m1_bb_pts,)#output_path+'S'+str(sec_num)+'_vM1')
            #sitk.WriteImage(cropped_im,output_path+'{}_01_cropped.tif'.format(sec_num))
        
        #else:
        if  m1_bb_pts is not None:
            sitk.WriteImage(sitk.GetImageFromArray(npim),output_path+'S{}_rabies_binary.tif'.format(sec_num))
            marked_im = mark_binary_im(sitk.GetImageFromArray(npim))
            sitk.WriteImage(marked_im,output_path+'S{}_rabies_boundary_marked.tif'.format(sec_num))
            
            masked_im = sitk.Mask(im,sitk.GetImageFromArray(npim))
            
            sitk.WriteImage(masked_im,output_path+'S{}_rabies_masked.tif'.format(sec_num))
    
    

100
None
101
None
102
None
103
None
104
None
105
None
106
None
107
None
108
None
109
None
10
None
110
None
111
None
112
None
113
None
114
None
115
None
116
None
117
None
118
None
119
None
11
None
120
None
12
None
13
None
14
None
15
None
16
None
17
None
18
None
19
None
1
None
20
None
21
None
22
None
23
None
24
[[5252, 1627, 0], [5461, 1807, 0]]
5252 5461 1627 1807
25
[[5884, 1479, 0], [6229, 1821, 0]]
5884 6229 1479 1821
26
[[6191, 1586, 0], [6638, 2056, 0]]
6191 6638 1586 2056
27
[[6461, 1513, 0], [7002, 2069, 0]]
6461 7002 1513 2069
28
[[5573, 2479, 0], [6201, 3109, 0]]
5573 6201 2479 3109
29
[[6614, 2380, 0], [7325, 3039, 0]]
6614 7325 2380 3039
2
None
30
[[6190, 2129, 0], [6943, 2839, 0]]
6190 6943 2129 2839
31
[[6396, 2011, 0], [7152, 2737, 0]]
6396 7152 2011 2737
32
[[7416, 2273, 0], [8210, 2964, 0]]
7416 8210 2273 2964
33
[[6650, 2026, 0], [7421, 2741, 0]]
6650 7421 2026 2741
34
[[6495, 2831, 0], [7257, 3553, 0]]
6495 7257 2831 3553
35
[[5613, 2848, 0], [6348, 3585, 0]]
5613 6348

In [2]:
landmarks_file = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Landmarks/MG50_lhs_cortical_landmarks.LandmarkAscii.landmarkAscii'

l = Landmarks(landmarks_file)

sg = AmiraSpatialGraph(generic_graph=True)
for pt in l.pts:
    sg.graph_data.add_edge(pt,pt)
    
    

In [5]:
sg.write_spatial_graph('/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Landmarks/MG50_lhs_cortical_pts.am')

In [6]:
sg_path = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Landmarks/MG50_lhs_cortical_pts.am'
sg = AmiraSpatialGraph(sg_path,generic_graph=True)

pts = []
for edge in sg.graph_data.edge_list:
    pts.append(edge[0])
    
Landmarks(pts=pts).write_landmarks('/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Landmarks/MG50_lhs_vS1_Density_Filtered.landmarksAscii')